# Develop LSTM Models for Time Series Forecasting


Long Short-Term Memory networks, or LSTMs for short, can be applied to time series forecasting.

There are many types of LSTM models that can be used for each specific type of time series forecasting problem.

The objective of this notebook is to provide standalone example time series problem as a template that you can copy and adapt for your specific time series forecasting problem.

## Data

The LSTM model will learn a function that maps a sequence of past observations as input to an output observation. As such, the sequence of observations must be transformed into multiple examples from which the LSTM can learn.

Consider a given univariate sequence:

In [1]:
[10, 20, 30, 40, 50, 60, 70, 80, 90]

[10, 20, 30, 40, 50, 60, 70, 80, 90]

We can divide the sequence into multiple input/output patterns called samples, where three time steps are used as input and one time step is used as output for the one-step prediction that is being learned.

```
X,				y
10, 20, 30		40
20, 30, 40		50
30, 40, 50		60
...
```

The split_sequence() function below implements this behavior and will split a given univariate sequence into multiple samples where each sample has a specified number of time steps and the output is a single time step.

In [2]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


## Vanilla LSTM

A Vanilla LSTM is an LSTM model that has a single hidden layer of LSTM units, and an output layer used to make a prediction.

We can define a Vanilla LSTM for univariate time series forecasting as follows.

```
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
```

We are working with a univariate series, so the number of features is one, for one variable.

The number of time steps as input is the number we chose when preparing our dataset as an argument to the split_sequence() function.

The shape of the input for each sample is specified in the input_shape argument on the definition of first hidden layer.

We almost always have multiple samples, therefore, the model will expect the input component of training data to have the dimensions or shape:

```
[samples, timesteps, features]
```

Our split_sequence() function in the previous section outputs the X with the shape [samples, timesteps], so we easily reshape it to have an additional dimension for the one feature.

```
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
```

In this case, we define a model with 50 LSTM units in the hidden layer and an output layer that predicts a single numerical value.

The model is fit using the efficient Adam version of stochastic gradient descent and optimized using the mean squared error, or ‘mse‘ loss function.

```
# fit model
model.fit(X, y, epochs=200, verbose=0)
```

After the model is fit, we can use it to make a prediction.

We can predict the next value in the sequence by providing the input:

```
[70, 80, 90] -> [100]
```



In [11]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 4145.8389
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 4108.6216
Epoch 3/200
1/1 [==============================] - 0s 5ms/step - loss: 4072.1296
Epoch 4/200
1/1 [==============================] - 0s 4ms/step - loss: 4036.2493
Epoch 5/200
1/1 [==============================] - 0s 8ms/step - loss: 4000.9475
Epoch 6/200
1/1 [==============================] - 0s 9ms/step - loss: 3966.1758
Epoch 7/200
1/1 [==============================] - 0s 10ms/step - loss: 3931.8408
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 3897.8621
Epoch 9/200
1/1 [==============================] - 0s 7ms/step - loss: 3864.1570
Epoch 10/200
1/1 [==============================] - 0s 7ms/step - loss: 3830.8406
Epoch 11/200
1/1 [==============================] - 0s 6ms/step - loss: 3797.5842
Epoch 12/200
1/1 [==============================] - 0s 13ms/step - loss: 3764.2034
Epoch 13/200
1/1 [=====

1/1 [==============================] - 0s 10ms/step - loss: 17.6303
Epoch 102/200
1/1 [==============================] - 0s 9ms/step - loss: 16.0693
Epoch 103/200
1/1 [==============================] - 0s 10ms/step - loss: 14.6494
Epoch 104/200
1/1 [==============================] - 0s 8ms/step - loss: 13.3460
Epoch 105/200
1/1 [==============================] - 0s 8ms/step - loss: 12.1424
Epoch 106/200
1/1 [==============================] - 0s 9ms/step - loss: 11.0315
Epoch 107/200
1/1 [==============================] - 0s 10ms/step - loss: 10.0159
Epoch 108/200
1/1 [==============================] - 0s 9ms/step - loss: 9.1059
Epoch 109/200
1/1 [==============================] - 0s 9ms/step - loss: 8.3154
Epoch 110/200
1/1 [==============================] - 0s 15ms/step - loss: 7.6567
Epoch 111/200
1/1 [==============================] - 0s 9ms/step - loss: 7.1345
Epoch 112/200
1/1 [==============================] - 0s 7ms/step - loss: 6.7402
Epoch 113/200
1/1 [========================

In [4]:
print(X)

[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]

 [[60]
  [70]
  [80]]]


## Prediction

The model expects the input shape to be three-dimensional with [samples, timesteps, features], therefore, we must reshape the single input sample before making the prediction.

In [12]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.88749]]


In [13]:
# demonstrate prediction
x_input = array([80, 90, 100])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[116.53066]]


## Stacked LSTM

Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as a Stacked LSTM model.

An LSTM layer requires a three-dimensional input and LSTMs by default will produce a two-dimensional output as an interpretation from the end of the sequence.

We can address this by having the LSTM output a value for each time step in the input data by setting the return_sequences=True argument on the layer. This allows us to have 3D output from hidden LSTM layer as input to the next.

We can therefore define a Stacked LSTM as follows.

In [9]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.5506]]
